In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

from keras.utils import np_utils

import warnings 
warnings.filterwarnings(action='ignore')

In [ ]:
import numpy as np      
import pandas as pd       
import re
import matplotlib.pyplot as plt   
import seaborn as sns
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix


import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import SimpleRNN, LSTM, Embedding, Dense

from keras.utils import np_utils


os.chdir('/content/drive/MyDrive/Colab Notebooks/KUBIG_2021_Spring/경진대회')

In [ ]:
# Sklearn tools
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Keras preprocessing, models, evaluators
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Activation, Dropout, GlobalMaxPool1D, Conv1D
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from keras.preprocessing import text, sequence
from keras import utils
from keras.models import load_model


### 하이퍼파라미터 정의

In [ ]:
model_name = 'transformer'
BATCH_SIZE = 64
max_len = 96
EPOCHS = 10

In [ ]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

## Data 불러오기

In [ ]:
path = './data_in/'

train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test_x.csv')
sample_submission = pd.read_csv(path+'sample_submission.csv')

In [ ]:
train.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [ ]:
test.head()

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...


In [ ]:
# 부호 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

# 불용어 제거해주는 함수
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
# stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
#              "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
#              "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
#              "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
#              "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
#              "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
#              "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
#              "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
#              "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
#              "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
#              "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

stopwords = list(set(stopwords.words('english')))

# 소문자로 바꾼 후 전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [ ]:
train.head()

,index,text,author
0,0,almost choking much much wanted say strange ex...,3
1,1,sister asked suppose,2
2,2,engaged one day walked perusing janes last let...,1
3,3,captain porch keeping carefully way treacherou...,4
4,4,mercy gentlemen odin flung hands dont write an...,3


In [ ]:
# 먼저 train 데이터와 test 데이터 인덱스 없이 배열로 만들기
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
Y_train = np.array([x for x in train['author']])

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)

(54879,)
(19617,)
(54879,)


In [ ]:
# Tokenizer
from keras.preprocessing.text import Tokenizer
# vocab_size = 2000  

# tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<OOV>")  

tokenizer = Tokenizer()
  # Tokenizer 는 데이터에 출현하는 모든 단어의 개수를 세고 빈도 수로 정렬해서 
  # num_words 에 지정된 만큼만 숫자로 반환하고, 나머지는 0 으로 반환합니다                 
tokenizer.fit_on_texts(X_train) # Tokenizer 에 데이터 실제로 입력
sequences_train = tokenizer.texts_to_sequences(X_train)    # 문장 내 모든 단어를 시퀀스 번호로 변환
sequences_test = tokenizer.texts_to_sequences(X_test)      # 문장 내 모든 단어를 시퀀스 번호로 변환

print(len(sequences_train), len(sequences_test))

54879 19617


In [ ]:
# 변환된 시퀀스 번호를 이용해 단어 임베딩 벡터 생성
word_index = tokenizer.word_index

In [ ]:
# 독립변수 데이터 전처리
  ## 문장의 길이기 제각각이기 때문에 벡터 크기 다 다름
  ## 그러므로 최대 시퀀스 길이 크기(211) 만큼 넉넉하게 늘리고
  ## 패딩(padding) 작업을 통해 나머지 빈 공간을 0으로 채움
max_len = 150    # 위에서 그래프 확인 후 정함
padding_type='post'

train_x = pad_sequences(sequences_train, padding='post', maxlen=max_len)
test_x = pad_sequences(sequences_test, padding=padding_type, maxlen=max_len)

print(train_x.shape, test_x.shape)

(54879, 150) (19617, 150)


In [ ]:
# 종속변수 데이터 전처리
train_y = np_utils.to_categorical(Y_train) # Y_train 에 원-핫 인코딩
print(train_y)
print(train_y.shape)

[[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]
(54879, 5)


In [ ]:
# vocab_size = 2000
# oov_tok = "<OOV>"

In [ ]:
# # tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(X_train)

# sequences = tokenizer.texts_to_sequences(X_train)
# X_train = sequence.pad_sequences(sequences, maxlen = max_len)
# sequences = tokenizer.texts_to_sequences(X_valid)
# X_valid = sequence.pad_sequences(sequences, maxlen = max_len)

In [ ]:
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

In [ ]:
print(vocab_size)

47121


# Transformer classification

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
model = Sequential([TokenAndPositionEmbedding(max_len, vocab_size, embed_dim)])

## 모델링

In [ ]:
embed_dim = 128  # Embedding size for each token
num_heads = 6  # Number of attention heads
ff_dim = 128 # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(5, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

(None, 150, 128)
(None, 150, 128)
(None, 128)
(None, 128)


In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

checkpoint_path = "tmp_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

In [ ]:
# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=4)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)

checkpoint_path = DATA_OUT_PATH + model_name + '/weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

./data_out/transformer -- Folder already exists 



In [ ]:
# 계층 교차 검증
n_fold = 5  
seed = 42

cv = StratifiedKFold(n_splits = n_fold, shuffle=True, random_state=seed)

# 테스트데이터의 예측값 담을 곳 생성
test_y = np.zeros((test_x.shape[0], 5))


for i, (i_trn, i_val) in enumerate(cv.split(train_x, Y_train), 1):
    print(f'training model for CV #{i}')

    model.fit(train_x[i_trn], 
            to_categorical(Y_train[i_trn]),
            validation_data=(train_x[i_val], to_categorical(Y_train[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[cp_callback, earlystop_callback])
    
    test_y += model.predict(test_x) / n_fold    # 나온 예측값들을 교차 검증 횟수로 나눈다

training model for CV #1
Epoch 1/10
86/86 [==============================] - 37s 410ms/step - loss: 1.6485 - accuracy: 0.2478 - val_loss: 1.5412 - val_accuracy: 0.2783

Epoch 00001: val_loss improved from inf to 1.54118, saving model to ./data_out/transformer/weights.h5
Epoch 2/10
86/86 [==============================] - 35s 405ms/step - loss: 1.4388 - accuracy: 0.3594 - val_loss: 0.9573 - val_accuracy: 0.6184

Epoch 00002: val_loss improved from 1.54118 to 0.95729, saving model to ./data_out/transformer/weights.h5
Epoch 3/10
86/86 [==============================] - 35s 409ms/step - loss: 0.7803 - accuracy: 0.6988 - val_loss: 0.7112 - val_accuracy: 0.7359

Epoch 00003: val_loss improved from 0.95729 to 0.71118, saving model to ./data_out/transformer/weights.h5
Epoch 4/10
86/86 [==============================] - 35s 411ms/step - loss: 0.4681 - accuracy: 0.8303 - val_loss: 0.7802 - val_accuracy: 0.7263

Epoch 00004: val_loss did not improve from 0.71118
Epoch 5/10
86/86 [================

In [ ]:
# 테스트 데이터의 예측값 확인
test_y

array([[2.67048142e-05, 1.88803818e-02, 1.86408299e-01, 7.94635389e-01,
        4.92332751e-05],
       [5.27595690e-04, 8.36893484e-01, 8.04220650e-04, 1.50338043e-01,
        1.14366262e-02],
       [9.97954726e-01, 2.03340833e-03, 2.64972240e-06, 3.66281173e-06,
        5.54692300e-06],
       ...,
       [7.18415527e-06, 9.99981850e-01, 8.73855214e-07, 9.21624633e-06,
        8.67195096e-07],
       [1.14311487e-05, 9.99983281e-01, 1.13474303e-06, 3.32739117e-06,
        8.42542282e-07],
       [9.99948546e-01, 2.97889171e-05, 7.23938609e-06, 2.19515655e-06,
        1.22444611e-05]])

In [ ]:
# sample_submission 에 대입
sample_submission[['0','1','2','3','4']] = test_y
sample_submission

,index,0,1,2,3,4
0,0,0.000027,0.018880,1.864083e-01,0.794635,4.923328e-05
1,1,0.000528,0.836893,8.042207e-04,0.150338,1.143663e-02
2,2,0.997955,0.002033,2.649722e-06,0.000004,5.546923e-06
3,3,0.000021,0.000010,9.998060e-01,0.000004,1.581191e-04
4,4,0.793558,0.000502,2.487749e-04,0.205541,1.499311e-04
...,...,...,...,...,...,...
19612,19612,0.000108,0.999887,1.608204e-07,0.000004,8.802383e-07
19613,19613,0.001850,0.000003,4.724074e-05,0.000003,9.980965e-01
19614,19614,0.000007,0.999982,8.738552e-07,0.000009,8.671951e-07
19615,19615,0.000011,0.999983,1.134743e-06,0.000003,8.425423e-07


In [ ]:
sample_submission.to_csv("sample_submission.csv", index=False)